<a href="https://colab.research.google.com/github/TheOneTrueGuy/DigitalDayDream/blob/main/1simple_SDXL_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First the necessary installs and dependencies for using diffusers and transformers.

In [1]:
!pip install --upgrade diffusers transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.4 MB/s eta 0:00:00


In [2]:
#@title You gotta use this too if you want easy save to your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!wget "https://drive.google.com/file/d/196HuG4w9w_crcIjKaMGMxDme2R9UOS-F/view?usp=drive_link" -O "v2_lora_RollingClockwise.ckpt"

Otherwise you can just download the final zip or .mp4 file.

In [3]:
#@title imagemagick gives us access to a simple, fast zoom function
!apt install imagemagick

def zoom(imagename, zoomscale=1.1):
  os.system(f"convert {imagename} -distort SRT {zoomscale},0 -gravity center {imagename}")



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libdjvulibre-text libdjvulibre21
  libfftw3-double3 libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10
  libwmflite-0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng enscript gimp
  gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance sane-utils
  texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev inkscape
  poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fon

In [4]:
# run this first then make anim in next cell
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline
from diffusers.utils import load_image
#from diffusers import DiffusionPipeline
import datetime as dt
import time
import os
import random
from PIL import ImageFont, ImageDraw, Image

class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, False)


pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
  "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16
)


#pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#  "chillpixel/starlight-animated-sdxl", torch_dtype=torch.float16
#)
#chillpixel/starlight-animated-sdxl

# load lora moved to separate cell for optionality

pipe = pipe.to("cuda")
pipe.safety_checker = FakeSafety()
# the following might work with some param tweaking but I didn't have good luck with these LoRas
# Norod78/sdxl-pumpkin-head-lora #not good
# lora-library/tekakutli-dinosaurs #didn't work


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
#@title Load LoRa layer(s) this is optional, don't run if you just want to use default SDXL

#https://replicate.com/jakedahn/sdxl-isometric-geology
# v2_lora_RollingClockwise.ckpt

pipe.load_lora_weights("goofyai/cyborg_style_xl") # use with "cyborg" or "android" keyword
#pipe.load_lora_weights("datamonet/Makima_Chainsaw_Man_LoRA") #needs weight_name
#pipe.load_lora_weights("")

#pipe.load_lora_weights("jakedahn/sdxl-isometric-geology") #spare for swapping in new loras
#pipe.load_lora_weights("") #spare for swapping in new loras

#FFusion/FFusionXL-LoRa-SDXL-Potion-Art-Engine

# cadaeic/duelyst-landscape-lora
# Duskfallcrew/the-crystal-exarch-15
# datamonet/Makima_Chainsaw_Man_LoRA


In [6]:
#@title upload start image 512x512 will be quick and look good

import os

from google.colab import files
uploaded = files.upload()
name=uploaded.keys()
name = list(name)[0]
print(name)


Saving st-0299.png to st-0299.png
st-0299.png


In [ ]:

zip_name = "easy_to_remember_name5" #@param {type:"string"}
negprompt = " monochrome, dumb, flat, boring, ordinary" #@param {type:"string"}
prompt= "Many happy robots dancing in the mechanical jungle of gears, axles and vine like chains stretching from telephone poles and silver pillars" #@param {type:"string"}
zoom_scale=0.99 #@param {type:"number"}
# eventually we will add a math function to the zoom scale and/or keyframe triggers to change the rate.
# notice! zoom scale above 1 like 1.05 will zoom in, below 1, like 0.995 will zoom out
total_frames=300 #@param {type:"number"}

# needs zip_name check to see if file already exists, append random number to prevent overwrite
zip_name=zip_name+".zip"




outdir = dt.datetime.now().strftime("%Y%m%d%H%M")
# if outdir exists add 4 random digits to it:
if os.path.exists(outdir):
    # Generate 4 random digits
    random_digits = str(random.randint(1000, 9999))

    # Create a new output directory with the random digits appended
    new_outdir = outdir + random_digits

    os.mkdir(new_outdir)
    outdir=new_outdir
    print("Created new output directory:", new_outdir)
else:
    os.system(f"mkdir {outdir}")
print("date time now:" + outdir)

xz=0


total=total_frames



for xn in range(0,total,1):
  if xn%50==49: torch.cuda.empty_cache()
  init_image = load_image(name).convert("RGB")#

  image = pipe(
               prompt,
               num_inference_steps=10,
               strength=0.81,
               guidance_scale=7.34,
               negative_prompt = negprompt ,
               image=init_image
               ).images[0]
  outname= name

  out2=outdir+"/st-" + str(xz).zfill(4) + ".png"
  image.save(out2) # save animation frame
  xz+=1
  image.save(outname)
  # Check if zoom_scale is not equal to zero before calling the zoom function
  if zoom_scale != 0:
    zoom(outname, zoom_scale)

  #next 2 lines apply motion distortion from moto*.txt motion files
  # deleted lines to be replaced soon

# all done, zip the files and save them
os.system(f"zip -r {zip_name} {outdir}")
os.system(f"cp {zip_name}  /content/drive/MyDrive")


date time now:202310090447


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
# ffmpeg call to make .mp4 will go here soon

In [ ]:
#run this cell after starting cell(s) above if you don't want to burn
#google compute credits unnecessarily.
from google.colab import runtime
runtime.unassign()